# Project 1, Part 2, Customer Related Queries



# Included Modules and Packages

Code cell containing your includes for modules and packages

In [4]:
import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  Remember you can use any code from the labs.

In [5]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [6]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

# 1.2.1 Total Number of Customers for all of AGM

Each record in the customer table is an individual customer.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return only 1 row into a Pandas dataframe and should look similar to this: 

||total_number_of_customers|
|---|---|
|0|31082|

In [14]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select count(customer_id) as total_number_of_customers
from customers
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,total_number_of_customers
0,31082


In [22]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select *
from zip_codes
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,zip,latitude,longitude,city,state,population,area,density,time_zone
0,08074,39.7158,-75.1640,Richwood,NJ,15,0.0886,169.39,America/New_York
1,08240,39.4873,-74.5318,Pomona,NJ,2293,1.5196,1508.93,America/New_York
2,08876,40.5880,-74.6874,Somerville,NJ,22059,15.1172,1459.20,America/New_York
3,10001,40.7506,-73.9972,New York,NY,22924,0.6675,34341.44,America/New_York
4,32026,30.0541,-82.1815,Raiford,FL,1907,0.6333,3011.38,America/New_York
...,...,...,...,...,...,...,...,...,...
32718,47367,40.0827,-85.3872,Oakville,IN,23,0.0866,265.47,America/Indiana/Indianapolis
32719,63079,38.2606,-91.0998,Stanton,MO,24,0.3523,68.12,America/Chicago
32720,63738,37.0893,-89.9574,Brownwood,MO,31,0.1171,264.70,America/Chicago
32721,68954,40.6227,-98.2374,Inland,NE,14,1.7437,8.03,America/Chicago


# 1.2.2 Total Number of Customers by Store

Each record in the customer table is an individual customer.  

Use the customer's closest_store_id to decide which store they belong to.

For store_name use the store's city.

Sort by store_name in alphabetical order.

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return 5 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||store_name|total_number_of_customers|
|---|---|---|
|0|Berkeley|8138|
|...|...|...|
|4|Seattle|7214|

In [17]:
rollback_before_flag = True
rollback_after_flag = True

query = """
select s.city as store_name,
       count(c.customer_id) as total_number_of_customers
from stores s
     join customers as c
     on s.store_id = c.closest_store_id
group by store_name
order by store_name
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,store_name,total_number_of_customers
0,Berkeley,8138
1,Dallas,6359
2,Miami,5725
3,Nashville,3646
4,Seattle,7214


# 1.2.3 List of Customers who have signed up but not bought anything

Each record in the customer table is an individual customer.  

Find all customers who are in the customers table but don't have any sales in the sales table.  

Sort by customer last_name, then first_name

Write 1 and only 1 query.  Note that the query may have as many subqueries, including "with" clauses, as you wish.  

Name column headers exactly as shown in the example below. 

Format data exactly as shown in the example below.

Ensure that when you check this Juptyer Notebook into GitHub that the query results in the Pandas dataframe are clearly visible in GitHub.

The query should return 35 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||last_name|first_name|
|---|---|---|
|0|Agott|Tracy|
|...|...|...|
|34|Vedekhin|Cyrill|

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select c.last_name, c.first_name
from customers as c
where c.customer_id not in (select distinct customer_id from sales)
order by last_name, first_name
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,last_name,first_name
0,Agott,Tracy
1,Arnke,Daniella
2,Assandri,Hyacintha
3,Borman,Felice
4,Breit,Domini
5,Butterick,Jacenta
6,Camillo,Marysa
7,Dukelow,Lilas
8,Dukesbury,Corinna
9,Ellaway,Lorianna


# 1.2.4 What is the percentage of customers per population at the zip code level?

Each record in the customer table is an individual customer.  

Use the customer's zip code (not the store's zip code). 

Use the zip_code table in the secondary dataset to find the population in that zip code.

Sort by highest percentage first.  Note that we are rouding to 3 decimal place for display, but note that you may need to sort on values that are not rounded.

**Note: When a query result has a large number of rows, Pandas will only display the first 5 rows, a row with ellipses, and the last 5 rows. This is ok.**

**Note: the reference output is in Markdown which drops trailing zeros.  Pandas does not drop trailing zeros.  This is ok.**

The query should return 550 rows into a Pandas dataframe. The first and last rows should look similar to this: 

||zip|percentage_customers_per_population|
|---|---|---|
|0|98164|1.29|
|...|...|...|
|549|75034|0.001|

In [32]:
rollback_before_flag = True
rollback_after_flag = True

query = """
with

  subquery_number_of_customers as 
    (
       select zip,
       count(customer_id) as total_number_of_customers
       from customers
       group by zip
    ),
    
  subquery_population as
    (
       select zip,
       population
       from zip_codes
    )
    
select n.zip,
       n.total_number_of_customers/p.population as percentage_customers_per_population

from subquery_number_of_customers n
     join subquery_population p
       on n.zip = p.zip

order by percentage_customers_per_population DESC
"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,zip,percentage_customers_per_population
0,98164,0.012903
1,98050,0.010870
2,33109,0.010526
3,94613,0.010453
4,37240,0.010283
...,...,...
545,33033,0.000017
546,75067,0.000015
547,75035,0.000014
548,94565,0.000010
